# Class 23: Intro to Machine Learning

Plan for today:
- Creating confidence intervals
- Introduction to Machine Learning


## Notes on the class Jupyter setup

If you have the *ydata123_2024a* environment set up correctly, you can get the class code using the code below (which presumably you've already done given that you are seeing this notebook).  

In [3]:
import YData

# YData.download.download_class_code(23)   # get class code    
# YData.download.download_class_code(23, TRUE) # get the code with the answers 


There are also similar functions to download the homework:

In [4]:
# YData.download.download_homework(9)  # downloads the homework 

If you are using colabs, you should install the YData packages by uncommenting and running the code below.

In [5]:
# !pip install https://github.com/emeyers/YData_package/tarball/master

If you are using google colabs, you should also uncomment and run the code below to mount the your google drive

In [6]:
# from google.colab import drive
# drive.mount('/content/drive')

In [7]:
import statistics
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
from urllib.request import urlopen

import matplotlib.pyplot as plt
%matplotlib inline

## 1. Using hypothesis tests to generate confidence intervals

There are several methods we that can be used to calculate confidence intervals, including using a computational method called the "bootstrap" and using "parametric methods" that involve using probability distributions. If you take a traditional introductory statistics class you will learn some of these methods.

Below we use a less conventional method to calculate confidence intervals by looking at all parameters values that a hypothesis test fails to reject (at the p-value < 0.05 level). As you will see, the method gives similar results to other methods, although it requires a bit more computation time.

As an example, let's create a confidence interval for the population proportion of movies $\pi$ that pass the Bechdel test. As is the case for all confidence intervals, this confidence interval gives a range of plausible values that likely contains the true population proportion $\pi$.


In [8]:
# To start, let's use a function that generates a statistic p-hat that is consistent with a particular population parameter value pi

def generate_prop_bechdel(n, null_prop):
    
    random_sample = np.random.rand(n) <= null_prop
    return np.mean(random_sample)


# Call the function to get one p-value consistent with H0: pi = .5

generate_prop_bechdel(1794, .5)


0.4983277591973244

In [9]:
# The function below calculates a p-value for the Bechdel data based on a particular pi value that is specified in a null hypothesis.
# (i.e., it is a function that encapsulates the hypothesis test you ran in class 20).


def get_Bechdel_pvalue(null_hypothesis_pi, plot_null_dist = False):
    
    
    # Step 2: The observed p-hat value
    prop_passed = 803/1794
    
    
    # Step 3: Generate the null distribution 
    null_dist = []
    
    for i in range(10000):    
        null_dist.append(generate_prop_bechdel(1794, null_hypothesis_pi))
    
    
    # Step 4: Calculate a "two-tailed" p-value which is the proportion of statistcs
    # more extreme than the observed statistic
    
    statistic_deviation = np.abs(null_hypothesis_pi - prop_passed)
    
    pval_left = np.mean(np.array(null_dist) <= null_hypothesis_pi - statistic_deviation)
    pval_right = np.mean(np.array(null_dist) >= null_hypothesis_pi + statistic_deviation)
    
    p_value = pval_left + pval_right

    
    # plot the null distribution and lines indicating values more extreme than the observed statistic 
    if plot_null_dist:
        
        plt.hist(null_dist, edgecolor = "black", bins = 30);
        plt.axvline(null_hypothesis_pi - statistic_deviation, color = "red");
        plt.axvline(null_hypothesis_pi + statistic_deviation, color = "red");
        plt.axvline(null_hypothesis_pi, color = "yellow");

        
        plt.title("Pi-null is: " + str(null_hypothesis_pi) + "      "  +
                  "p-value is: " + str(round(p_value, 5)))
      
    # return the p-value
    return p_value
    


In [10]:
# test the function with the value H0: pi = .5  (as we did in class 20)



# test the function with the value H0: pi = .45





In [14]:
# create a range range of H0: pi = x  values

possible_null_pis = np.round(np.arange(.4, .5, .005), 5)

possible_null_pis  



array([0.4  , 0.405, 0.41 , 0.415, 0.42 , 0.425, 0.43 , 0.435, 0.44 ,
       0.445, 0.45 , 0.455, 0.46 , 0.465, 0.47 , 0.475, 0.48 , 0.485,
       0.49 , 0.495])

In [15]:
%%time


# get the p-value for a range of H0: pi = x  values

pvalues = []

for null_pi in possible_null_pis:
    
    curr_pvalue = get_Bechdel_pvalue(null_pi)
    
    pvalues.append(curr_pvalue)





CPU times: user 3.6 s, sys: 10.5 ms, total: 3.61 s
Wall time: 3.62 s


In [16]:
# view the p-values 
# convention calls a p-value < 0.05 is "statistically significant" indicating a pi imcompatible with the null hypothesis
# our confidence interval is all pi values that are not statistically significant (i.e., pi values that are consistent with particular H0)

pvalue_df = pd.DataFrame({"pi": possible_null_pis, 
                         "p-value": pvalues,
                         "plausible": np.array(pvalues) > .05})


pvalue_df

,pi,p-value,plausible
0,0.400,0.0000,False
1,0.405,0.0002,False
2,0.410,0.0017,False
3,0.415,0.0050,False
4,0.420,0.0186,False
5,0.425,0.0555,True
6,0.430,0.1338,True
7,0.435,0.2868,True
8,0.440,0.5265,True
9,0.445,0.8326,True


In [ ]:
# plot p-values as as function of H0 pi's

sns.relplot(pvalue_df, x = "pi", y = "p-value");

plt.axhline(.05, color = "red");


In [ ]:
# Get all plausible Pi values

plausible_pis = possible_null_pis[np.array(pvalues) >= .05]

plausible_pis

In [ ]:
# get the CI as the max and min plausible pi values 

(min(plausible_pis), max(plausible_pis))


In [ ]:
# using the statsmodels package to compute a confidence interval for a proportion

import statsmodels.api as sm

CI_lower, CI_upper = sm.stats.proportion_confint(803, 1794, alpha = .05, method = "normal")

(round(CI_lower, 3), round(CI_upper, 3))

# 95% CI

In [ ]:
CI_lower, CI_upper = sm.stats.proportion_confint(803, 1794, alpha = .00000001, method = "normal")
(round(CI_lower, 3), round(CI_upper, 3))

# 99% CI

## 2. Intro to Machine Learning:  Features (X) and labels (y)

In supervised machine learning, we use a computer algorithm called a "pattern classifier" to learn relationships between a set of features X, and a label y. When the classifier is given new examples X, it can then make new predictions y. 


In [ ]:
penguins = sns.load_dataset("penguins")

penguins = penguins.dropna()

penguins = penguins.sample(frac = 1)

penguins.head()

In [18]:
# Let's explore how many different members there are of each species in our data set? 





#### Questions: 

1. If we had to guess the species of the penguin without knowing any of the penguin's features, species of penguin should we guess? 
A: Always guess Adelie


2. If we were to following the optimal guessing strategy, what percent of our guess would be correct (i.e., what would our classification accuracy be)? 


In [17]:
# get proportion that are a particular species





To begin the classification process, let's store the features (X) and the labels (y) in separate names called `X_penguin_features` and `y_penguin_labels` respectively. 

In [ ]:
# get the features and the labels






## 3. k-Nearest Neighbors classifier

To explore classification, let's use a k-Nearest Neighbors classifier to predict the species of a penguin based on particular features the penguin has such as the penguin's bill length and body mass. 

Let's construct a K-Nearest Neighbor classifier (KNN) using 5 neighbors for predictions (i.e., k = 5 so we are using a 5-Nearest Neighbor classifier). 

We can do this using the `KNeighborsClassifier(n_neighbors = )` function.  



In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Construct a classifier a 5 nearest neighbor classifier



Let's now train the classifier (the KNN classifier just stores the data during training)


In [ ]:
# “train” the classifier (which for a KNN classifier just involves memorizing the training data)




Let's now use the classifier to make predictions

In [ ]:
# make predictions





Let's get the prediction (classificaton accuracy) which is the proportion of predictions that are correct

In [ ]:
# get the classification accuracy




Let's repeat our analysis with k = 1 to see what happens...

In [ ]:
# What happens if k = 1?


# construct a classifier



# “train” the classifier (which for a KNN classifier just involves memorizing the training data)



# make predictions



# get classification accuracy



Do we believe we have a perfect classifier???


## 4. Cross-validation

To avoid over-fitting, we need to split our data into a training and test set. 

The classifier "learns" the relationship between features (X) and labels (y) on the **training set**.

The classifier makes predictions on the features (X) of the **test set**. 

We compare the classifier's predictions on the test features (X) to the actual labels y, to get a more accuracy assessment of the **classification accuracy**.


Let's try this now...



In [ ]:
# manually create a training with 250 examples, and a test set that has the rest of the data









In [ ]:
from sklearn.model_selection import train_test_split

# split data into a training and test set








In [ ]:
from sklearn.neighbors import KNeighborsClassifier


# construct a classifier



# “train” the classifier (which for a KNN classifier just involves memorizing the training data)




In [ ]:
# get the predictions





In [ ]:
# Get the prediction accuracy 




In [ ]:
# Test the classifier on the test set using the .score() method

# prediction accuracy on the test set





In [ ]:
# What happens if we test the classifier on the training set? 





### K-fold cross-validation

In k-fold cross-validation we split our data into k-parts (note, the k here has no relation to the k in k-Nearest Neighbor - it is just that k is a frequent letter to use in math to denote integer values).  

To run a k-fold cross-validation analysis, we train the classifier on k-1 parts of the data and test it on the remaining part. We repeat this process k times to get k classification accuracies. We then take the average of these results as our estimate of our overall classification accuracy. 

We can use the scikit-learn `cross_val_score()` to easily do this...


In [ ]:
from sklearn.model_selection import cross_val_score


# construct knn classifier



# do 5-fold cross-validation






Next class, buliding a KNN classifier ourselves...